In [16]:
import torch
import wandb
import os


# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

In [ ]:
wandb.login()

## Gaussian Splatting without Camera Parameter Calibration Images

In [18]:
name_dir_data = "cam_params_big_brett"
val_pic_list = ["val.jpg"]

In [19]:
iterations = 19050
eval_interval  = 50
sh_degree = 3
percent_dense = 0.005
densification_interval = 150
densify_from_iter = 50
densify_until_iter = 25000
densify_grad_threshold = 0.001
lambda_dssim  = 0.5

In [ ]:
#W&B
wandb.init(
    project="gaussian-splatting",
    name="test",
    config={
        "name": "test",
    }
)
config = wandb.config


In [21]:
source_path = f"data/{name_dir_data}"
val_path_np    = os.path.join(source_path, "sparse", "0")
model_path_wo_p = f"outputs/{name_dir_data}_model"
valfilenames_str = " ".join(val_pic_list)

test_iterations = list(range(eval_interval, iterations+1, eval_interval))

if test_iterations[-1] != iterations:
    test_iterations.append(iterations)

test_iters_str = " ".join(str(i) for i in test_iterations)

In [ ]:
!python utils/make_val_file.py $val_path_np {valfilenames_str}
!python utils/sfm.py --source_path {source_path}

%run train.py --source_path $source_path --model_path $model_path_wo_p --iterations $iterations --sh_degree $sh_degree --percent_dense $percent_dense --densification_interval $densification_interval --densify_from_iter $densify_from_iter --densify_until_iter $densify_until_iter --densify_grad_threshold $densify_grad_threshold --lambda_dssim $lambda_dssim --disable_viewer --eval --test_iterations $test_iters_str

In [ ]:

%run utils/render.py --source_path $source_path --model_path $model_path_wo_p --resolution 1080 --iteration $iterations


In [ ]:
%run utils/render_new_cams.py --model_path $model_path_wo_p --iters $iterations --scale 1.0 --steps 5

## Gaussian Splatting with Camera Parameter Calibration Images

In [59]:
input_path_params_dir = "cam_params_big_brett"
name_dir_data = "moritz_without_brett"
val_pic_list = ["val.jpg"]


In [60]:
iterations = 1100
eval_interval  = 50
sh_degree = 4
percent_dense = 0.0050288222595324388
densification_interval = 95
densify_from_iter = 150
densify_until_iter = 1000
densify_grad_threshold = 0.00095380906898316
lambda_dssim  = 0.21222436138314285
depth_l1_weight_final = 1.9585141792568015
depth_l1_weight_init = 0.10818129865381582

In [ ]:

wandb.init(
    project="gaussian-splatting",
    name="moritz_final",
    config={
        "name": "moritz_final",
    }
)
config = wandb.config


In [62]:


data_dir = "data"

source_path_params     = os.path.join(data_dir, input_path_params_dir)
input_path_params      = os.path.join(source_path_params, "distorted", "sparse", "0")
source_path_w_p        = os.path.join(data_dir, name_dir_data)
val_path               = os.path.join(source_path_w_p, "sparse", "0")
output_path_txt        = os.path.join(source_path_w_p, "model_txt")
outputs_dir            = "outputs"
model_path_w_p         = os.path.join(outputs_dir, f"{name_dir_data}_model")
txt_path               = os.path.join(output_path_txt, "images.txt")

os.makedirs(output_path_txt, exist_ok=True)

valfilenames_str = " ".join(val_pic_list)

depth_dir = os.path.join(source_path_w_p, "depth")

In [ ]:
!python utils/make_val_file.py $val_path {valfilenames_str}

!python utils/sfm.py --source_path {source_path_params}

!python utils/bin_to_txt.py -i {input_path_params} -o {output_path_txt}

!python utils/update_images_txt.py --original {txt_path} --output {txt_path}    

%run utils/sfm_with_params.py --source_path $source_path_w_p
%env TORCHDYNAMO_DISABLE=1

test_iterations = list(range(eval_interval, iterations+1, eval_interval))

if test_iterations[-1] != iterations:
    test_iterations.append(iterations)

test_iters_str = " ".join(str(i) for i in test_iterations)

%run utils/make_depth_scale.py --base_dir $source_path_w_p --depths_dir $depth_dir

%run train.py --source_path $source_path_w_p --model_path $model_path_w_p --iterations $iterations --sh_degree $sh_degree --percent_dense $percent_dense --densification_interval $densification_interval --densify_from_iter $densify_from_iter --densify_until_iter $densify_until_iter --densify_grad_threshold $densify_grad_threshold --lambda_dssim $lambda_dssim --disable_viewer --eval --test_iterations $test_iters_str --depths depth --mask_folder "segmantation" --antialiasing --white_background --depth_l1_weight_final $depth_l1_weight_final --depth_l1_weight_init $depth_l1_weight_init 

In [ ]:
%run utils/render.py --source_path $source_path_w_p --model_path $model_path_w_p --resolution 1080 --iteration $iterations --antialiasing

In [65]:
%run utils/render_new_cams.py --model_path $model_path_w_p --iters $iterations --scale 1.0 --steps 20 --antialiasing